# Notebook for training NN to reconstruct energy

In [1]:
import torch
import numpy as np
import pandas as pd
import uproot as up
from reco import process_data_energy_reco,Predictor,train,plot_predictions,evaluate_model,shuffle_segment_data,plot_loss_predictions
from IPython.display import clear_output
import matplotlib.pyplot as plot

Using device cuda:0


In [2]:
# Get device to be used
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
in_dim = 28
out_dim = 1
num_epochs = 1
batch_size = 100

data_path = "/cwork/rck32/eic/work_eic/root_files/August_7/sector_scint/run_1_n_0_8_10GeV_theta_90_500kevents.edm4hep.root:events"
inputs = process_data_energy_reco(data_path)

train_data, test_data, val_data = shuffle_segment_data(inputs)

Event #468218


In [ ]:
model = Predictor(input_size=56, num_classes=1, hidden_dim = 512, num_layers = 20)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5, weight_decay=1e-5)

In [ ]:
loss_hist = train(model, train_data,optimizer,device, num_epochs = 4, batch_size = 50, show_progress = True)

In [ ]:
plot_loss_predictions(loss_hist)

In [ ]:
len(test_data)

In [ ]:
mae, mse, rmse, r2 = evaluate_model(model, test_data, device)